In [2]:
import networkx as nx
import sqlite3
import pathSearch, db
from tqdm.notebook import tqdm  # ノートブックではこちらが見やすい

dbpath = "paths_dict.db"

# 1. 領域指定
points = [
    [35.20902036505841, 136.86844825744632],
    [35.140265534798395, 137.02620506286624]
]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)

# 2. 道路データ取得（link, length形式の取得）
link, length = db.getRectangleRoadData(y1, x1, y2, x2)

# 3. NetworkX用グラフ作成
G = pathSearch.linkToGraph(link, length)

# 4. SQLiteのテーブル用意
with sqlite3.connect(dbpath) as conn:
    cur = conn.cursor()
    # テーブルとインデックス作成（進捗検索にも速い）
    cur.execute('''
        CREATE TABLE IF NOT EXISTS paths (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            coord1 TEXT,
            coord2 TEXT,
            length REAL
        )
    ''')
    cur.execute('CREATE INDEX IF NOT EXISTS idx_coord1 ON paths(coord1)')
    cur.execute('CREATE INDEX IF NOT EXISTS idx_coord2 ON paths(coord2)')
    cur.execute('CREATE INDEX IF NOT EXISTS idx_pair ON paths(coord1, coord2)')
    conn.commit()

    # 5. 最短経路長の計算・DB登録
    nodes = list(G.nodes)
    for source in tqdm(nodes, desc="Calculating shortest paths"):
        lengths = nx.single_source_dijkstra_path_length(G, source, weight='length')
        coord1 = str(source)
        # DBにまとめて書き込む
        records = [
            (coord1, str(target), dist)
            for target, dist in lengths.items() if target != source
        ]
        cur.executemany(
            'INSERT INTO paths (coord1, coord2, length) VALUES (?, ?, ?)',
            records
        )
    # withブロックで自動commit

Calculating shortest paths:   0%|          | 0/23338 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
import os
os.remove("paths.db")

PermissionError: [WinError 32] プロセスはファイルにアクセスできません。別のプロセスが使用中です。: 'paths.db'